In [11]:
### Este script consolida los datos de Happiness para cada varible por año en un solo archivo CSV

import os
import pandas as pd
import json

# Obtener la ruta del directorio del notebook actual
current_dir = os.getcwd()

# Construir la ruta relativa a la carpeta 'datos'
folder_path = os.path.join(current_dir, '../data')

# Define los mapeos de columnas
column_mappings = {
    'Country': ['Country', 'Country or region','Country name'],
    'Score': ['Happiness Score', 'Score', 'Happiness.Score','Ladder score','Happiness score'],
    'GDP per capita': ['Economy (GDP per Capita)', 'Economy..GDP.per.Capita.', 'GDP per capita','Logged GDP per capita','Explained by: GDP per capita'],
    'Family': ['Family', 'Social support','Explained by: Social support'],
    'Health (Life Expectancy)': ['Health (Life Expectancy)', 'Health..Life.Expectancy.', 'Healthy life expectancy','Explained by: Healthy life expectancy'],
    'Freedom': ['Freedom', 'Freedom to make life choices','Explained by: Freedom to make life choices'],
    'Perceptions of corruption': ['Trust (Government Corruption)', 'Trust..Government.Corruption.', 'Perceptions of corruption','Explained by: Perceptions of corruption'],
    'Generosity': ['Generosity','Explained by: Generosity']
}

# Crear un diccionario inverso para los nombres alternativos
alternate_names_to_official = {}
with open(json_path, 'r', encoding='utf-8') as file:
    data_country = json.load(file)
    for country, info in data_country.items():
        for alt_name in info.get('AlternateNames', []):
            alternate_names_to_official[alt_name] = country

# Función para estandarizar los nombres de las columnas y eliminar las no deseadas
def standardize_columns(df):
    standardized_columns = {}
    for standard_col, possible_cols in column_mappings.items():
        for col in possible_cols:
            if col in df.columns:
                standardized_columns[col] = standard_col
                break
    df = df.rename(columns=standardized_columns)
    # Eliminar columnas que no están en los mapeos
    columns_to_keep = list(standardized_columns.values())
    df = df[columns_to_keep]
    return df

# Lista para almacenar los dataframes
dataframes = []

# DataFrame consolidado inicial
consolidated_df = pd.DataFrame()

# Iterar sobre los archivos en la carpeta
for filename in os.listdir(folder_path):
    if filename.startswith('Happiness') and filename.endswith('.csv'):
        # Extraer el año del nombre del archivo
        year = int(''.join(filter(str.isdigit, filename)))
        
        # Leer el archivo CSV
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        
        # Eliminar filas vacías
        df.dropna(how='all', inplace=True)
        
        # Estandarizar las columnas y eliminar las no deseadas
        df = standardize_columns(df)
                
        # Asegurarse de que los valores de 'Country' sean cadenas y manejar valores nulos
        df['Country'] = df['Country'].astype(str).fillna('')
        
        # Reemplazar nombres alternativos por nombres oficiales
        df['Country'] = df['Country'].apply(lambda x: alternate_names_to_official.get(x, x))
        
        # Filtrar las filas basadas en el archivo JSON
        df = df[df['Country'].isin(data_country)]
        
        # Redondear las columnas numéricas a 3 dígitos si existen
        numeric_columns = ['Score', 'GDP per capita', 'Family', 
                           'Health (Life Expectancy)', 'Freedom', 
                           'Perceptions of corruption', 'Generosity']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = df[col].apply(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)
        
        # Añadir las columnas "Region" y "Continent" utilizando los datos del diccionario solo una vez
        if consolidated_df.empty:
            df['Region'] = df['Country'].apply(lambda x: data_country[x]['Region'] if x in data_country else '')
            df['Continent'] = df['Country'].apply(lambda x: data_country[x]['Continent'] if x in data_country else '')
            
            # Reordenar las columnas para que "Region" y "Continent" estén después de "Country"
            cols = df.columns.tolist()
            cols.insert(1, cols.pop(cols.index('Region')))
            cols.insert(2, cols.pop(cols.index('Continent')))
            df = df[cols]
        
        # Renombrar las columnas para incluir el año
        df = df.rename(columns={col: f"{col} ({year})" for col in df.columns if col not in ['Country', 'Region', 'Continent']})
        
        # Si el DataFrame consolidado está vacío, inicializarlo con el primer DataFrame
        if consolidated_df.empty:
            consolidated_df = df.set_index('Country')
        else:
            # Alinear los datos del nuevo DataFrame con el DataFrame consolidado
            df = df.set_index('Country')
            consolidated_df = consolidated_df.join(df, how='outer')
            
            # Añadir "Region" y "Continent" para nuevos países
            for country in df.index:
                if country not in consolidated_df.index:
                    consolidated_df.at[country, 'Region'] = data_country[country]['Region']
                    consolidated_df.at[country, 'Continent'] = data_country[country]['Continent']
                else:
                    if 'Region' not in consolidated_df.columns:
                        consolidated_df['Region'] = ''
                    if 'Continent' not in consolidated_df.columns:
                        consolidated_df['Continent'] = ''
                    consolidated_df.at[country, 'Region'] = data_country[country]['Region']
                    consolidated_df.at[country, 'Continent'] = data_country[country]['Continent']
        
# Ordenar el DataFrame combinado por el nombre de la columna "Country"
consolidated_df.sort_values(by=['Country'], inplace=True)

# Ordenar las columnas teniendo en cuenta el año
sorted_columns = sorted(consolidated_df.columns, key=lambda x: (x.split()[-1] if '(' in x else '', x))
consolidated_df = consolidated_df[sorted_columns]

# Guardar el dataframe consolidado en un nuevo archivo CSV
consolidated_df.to_csv('consolidado_happiness.csv')

print("Consolidación completa. Los datos consolidados se han guardado en consolidado_happiness.csv.")

Consolidación completa. Los datos consolidados se han guardado en consolidado_happiness.csv.
